## Change Directory to Main Katib folder

In [1]:
cd /root/katib/sdk/python

/root/katib/sdk/python


## Install Katib SDK

In [2]:
! python setup.py install

## Import Required Libraries

In [3]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container

In [4]:
import kubeflow.katib as kc
from kubeflow.katib import constants
from kubeflow.katib import utils
from kubeflow.katib import V1alpha3AlgorithmSetting
from kubeflow.katib import V1alpha3AlgorithmSetting
from kubeflow.katib import V1alpha3AlgorithmSpec
from kubeflow.katib import V1alpha3CollectorSpec
from kubeflow.katib import V1alpha3EarlyStoppingSetting
from kubeflow.katib import V1alpha3EarlyStoppingSpec
from kubeflow.katib import V1alpha3Experiment
from kubeflow.katib import V1alpha3ExperimentCondition
from kubeflow.katib import V1alpha3ExperimentList
from kubeflow.katib import V1alpha3ExperimentSpec
from kubeflow.katib import V1alpha3ExperimentStatus
from kubeflow.katib import V1alpha3FeasibleSpace
from kubeflow.katib import V1alpha3FileSystemPath
from kubeflow.katib import V1alpha3FilterSpec
from kubeflow.katib import V1alpha3GoTemplate
from kubeflow.katib import V1alpha3GraphConfig
from kubeflow.katib import V1alpha3Metric
from kubeflow.katib import V1alpha3MetricsCollectorSpec
from kubeflow.katib import V1alpha3NasConfig
from kubeflow.katib import V1alpha3ObjectiveSpec
from kubeflow.katib import V1alpha3Observation
from kubeflow.katib import V1alpha3Operation
from kubeflow.katib import V1alpha3OptimalTrial
from kubeflow.katib import V1alpha3ParameterAssignment
from kubeflow.katib import V1alpha3ParameterSpec
from kubeflow.katib import V1alpha3SourceSpec
from kubeflow.katib import V1alpha3Suggestion
from kubeflow.katib import V1alpha3SuggestionCondition
from kubeflow.katib import V1alpha3SuggestionList
from kubeflow.katib import V1alpha3SuggestionSpec
from kubeflow.katib import V1alpha3SuggestionStatus
from kubeflow.katib import V1alpha3TemplateSpec
from kubeflow.katib import V1alpha3Trial
from kubeflow.katib import V1alpha3TrialAssignment
from kubeflow.katib import V1alpha3TrialCondition
from kubeflow.katib import V1alpha3TrialList
from kubeflow.katib import V1alpha3TrialSpec
from kubeflow.katib import V1alpha3TrialStatus
from kubeflow.katib import V1alpha3TrialTemplate

## Define Experiment related Specs

In [5]:
algorithmsettings = V1alpha3AlgorithmSetting(
    name= "burn_in",
    value = "5"
    )
algorithm = V1alpha3AlgorithmSpec(
    algorithm_name = "bayesianoptimization",
    algorithm_settings = [algorithmsettings]
  )

# Metric Collector
collector = V1alpha3CollectorSpec(kind = "KerasEvent")
FileSystemPath = V1alpha3FileSystemPath(kind = "/train" , path = "Directory")
metrics_collector_spec = V1alpha3MetricsCollectorSpec(
    collector = collector,
    source = FileSystemPath)

# Objective
objective = V1alpha3ObjectiveSpec(
    goal = 1.3,
    objective_metric_name = "l2_loss",
    type = "minimize")

# Parameters

feasible_space_learning_rate = V1alpha3FeasibleSpace(min = "0.001", max = "0.002")
feasible_space_beta1 = V1alpha3FeasibleSpace(min = "0.88", max = "0.93")

parameters = [V1alpha3ParameterSpec(
    feasible_space = feasible_space_learning_rate, 
    name = "--learning_rate",
    parameter_type ="double"
    ),
    V1alpha3ParameterSpec(
    feasible_space = feasible_space_beta1, 
    name = "--beta1",
    parameter_type = "double"
    )]

# Trialtemplate
go_template = V1alpha3GoTemplate(
    raw_template =   "apiVersion: \"batch/v1\"\nkind: Job\nmetadata:\n  name: {{.Trial}}\n  namespace: {{.NameSpace}}\nspec:\n  template:\n    spec:\n      containers:\n      - name: {{.Trial}}\n        image: docker.io/johnugeorge/cnn-blerssi:0.3\n        command:\n        - \"python3\"\n        - \"/opt/blerssi-cnn-adam.py\"\n        {{- with .HyperParameters}}\n        {{- range .}}\n        - \"{{.Name}}={{.Value}}\"\n        {{- end}}\n        {{- end}}\n      restartPolicy: Never"
    )


trial_template= V1alpha3TrialTemplate(go_template=go_template)


# Experiment
experiment = V1alpha3Experiment(
    api_version="kubeflow.org/v1alpha3",
    kind="Experiment",
    metadata=V1ObjectMeta(name="blerssi-bayesopt",namespace="anonymous"),

    spec=V1alpha3ExperimentSpec(
         algorithm = algorithm,
         max_failed_trial_count=9,
         max_trial_count=12,
         objective = objective,
         parallel_trial_count=2,
         parameters = parameters ,
         trial_template = trial_template
    )
)

In [6]:
namespace = kc.utils.get_default_target_namespace()

In [7]:
kclient = kc.KatibClient()

## Create Experiment

In [8]:
kclient.create_experiment(experiment, namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-03-31T06:39:14Z',
  'generation': 1,
  'name': 'blerssi-bayesopt',
  'namespace': 'anonymous',
  'resourceVersion': '19224290',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/anonymous/experiments/blerssi-bayesopt',
  'uid': '5638e930-731a-11ea-bedf-42010aa00fec'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{'name': 'burn_in', 'value': '5'}]},
  'maxFailedTrialCount': 9,
  'maxTrialCount': 12,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 1.3,
   'objectiveMetricName': 'l2_loss',
   'type': 'minimize'},
  'parallelTrialCount': 2,
  'parameters': [{'feasibleSpace': {'max': '0.002', 'min': '0.001'},
    'name': '--learning_rate',
    'parameterType': 'double'},
   {'feasibleSpace': {'max': '0.93', 'min': '0.88'},
    'name': '--beta1',
    'parameterType': 'double'}],
  'trialTemplate': {'goTem

## Get Single Experiment Details

In [9]:
kclient.get_experiment(name="blerssi-bayesopt", namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-03-31T06:39:14Z',
  'finalizers': ['update-prometheus-metrics'],
  'generation': 1,
  'name': 'blerssi-bayesopt',
  'namespace': 'anonymous',
  'resourceVersion': '19224434',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/anonymous/experiments/blerssi-bayesopt',
  'uid': '5638e930-731a-11ea-bedf-42010aa00fec'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{'name': 'burn_in', 'value': '5'}]},
  'maxFailedTrialCount': 9,
  'maxTrialCount': 12,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 1.3,
   'objectiveMetricName': 'l2_loss',
   'type': 'minimize'},
  'parallelTrialCount': 2,
  'parameters': [{'feasibleSpace': {'max': '0.002', 'min': '0.001'},
    'name': '--learning_rate',
    'parameterType': 'double'},
   {'feasibleSpace': {'max': '0.93', 'min': '0.88'},
    'name': '--beta1',
    'paramet

## Get all Experiments

In [10]:
kclient.get_experiment()


{'apiVersion': 'kubeflow.org/v1alpha3',
 'items': [{'apiVersion': 'kubeflow.org/v1alpha3',
   'kind': 'Experiment',
   'metadata': {'creationTimestamp': '2020-03-30T09:15:16Z',
    'finalizers': ['update-prometheus-metrics'],
    'generation': 1,
    'name': 'bayesianoptimizationjob',
    'namespace': 'anonymous',
    'resourceVersion': '18661856',
    'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/anonymous/experiments/bayesianoptimizationjob',
    'uid': 'f82b3ae2-7266-11ea-bedf-42010aa00fec'},
   'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
     'algorithmSettings': [{'name': 'random_state', 'value': '10'}]},
    'maxFailedTrialCount': 3,
    'maxTrialCount': 5,
    'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
    'objective': {'goal': 0.9999,
     'objectiveMetricName': 'Validation-accuracy',
     'type': 'maximize'},
    'parallelTrialCount': 5,
    'parameters': [{'feasibleSpace': {'list': ['sgd', 'adam', 'ftrl']},
      'name': '--optimizer'

## Get experiment status

In [12]:
kclient.get_experiment_status(name="blerssi-bayesopt", namespace=namespace)


'Running'

## Check whether experiment has succeeded

In [13]:
kclient.is_experiment_succeeded(name="blerssi-bayesopt", namespace=namespace)

False

## Delete experiment

In [36]:
# kclient.delete_experiment(name="bayesianoptimization", namespace=namespace)#

## List Trials of an experiment

In [22]:
kclient.list_trials(name="blerssi-bayesopt", namespace=namespace)

[{'name': 'blerssi-bayesopt-42dql75j', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-5l7zfsp4', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-drgxzt7t', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-f57vtzjk', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-ffslt6g9', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-hj6cpcln', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-jpffc899', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-lvgm2xpn', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-nlcvwj5q', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-qcpc95qf', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-qqx94th7', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt-t5b6g5rp', 'status': 'Succeeded'}]

## List all experiments

In [16]:
kclient.list_experiments(namespace=namespace)


[{'name': 'bayesianoptimizationjob', 'status': 'Succeeded'},
 {'name': 'bayesianoptimizationjob1', 'status': 'Succeeded'},
 {'name': 'blerssi-bayesopt', 'status': 'Running'}]

In [23]:
kclient.get_optimal_hyperparmeters(name="blerssi-bayesopt", namespace=namespace)

{'status': 'Succeeded',
 'currentOptimalTrial': {'bestTrialName': 'blerssi-bayesopt-drgxzt7t',
  'observation': {'metrics': [{'name': 'l2_loss', 'value': 2.11414010362215}]},
  'parameterAssignments': [{'name': '--learning_rate',
    'value': '0.0016541997553521574'},
   {'name': '--beta1', 'value': '0.9167887608923336'}]}}